In [ ]:
#Install gspread library
pip install gspread

#Install gspred-dataframe library to work with pandas' DataFrames and Google Sheets
pip install gspread-dataframe

#Import the libraries
import gspread as gspread

import pandas as pd

import gspread_dataframe as gd

#Because the credentials that you gave me didn't work, I created a new one

#Get credentials to read the Google Sheet (They are in a JSON in my computer with the name: 'key.json')
gc = gspread.service_account(filename='key.json')

#Open the Google Sheet
sh=gc.open("Courier_Growth_Intern_1")

#Select the worksheet, get all values from that worksheet, and put them in a DataFrame using pandas
worksheet = sh.worksheet("RAW_DATA")
values=worksheet.get_all_values()
df=pd.DataFrame.from_records(values)

#Set the first row as the headers
df.columns = df.iloc[0]
#Select all the rows except the first one to create the DataFrame
df = df.iloc[1:]

#As I have the problem with the commas in the numbers, I transform the two columns that 
#I am gonna use to numeric and I replace the ',' for a '.'
#The coerce replace any non-numeric values with 'NaN'
df['dt_mins_1'] = pd.to_numeric(df['dt_mins_1'].str.replace(',', '.'), errors='coerce')
df['dt_mins_0'] = pd.to_numeric(df['dt_mins_0'].str.replace(',', '.'), errors='coerce')

#Filter by city and for when dt_mins_1 is greater than dt_mins_0. 
#Here I assume that dt_mins_0 is for the previous week of dt_mins_1
df_ALA=df[(df['city_code']=='ALA')]
df_final=df_ALA[(df_ALA['dt_mins_1'])>(df_ALA['dt_mins_0'])]

#Finally, I print the result in my Google Sheet in the worksheet 'Output'
gd.set_with_dataframe(sh.worksheet('Output'),df_final)